# Исследование надёжности заёмщиков

**Название проекта**

Исследование надёжности заёмщиков
_____
**Описание исследования.**
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
_____
**Цель исследования.**
Исследовать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  

_____
**Задачи исследования.**
* научиться прогнозировать вероятность оттока (на уровне следующего месяца) для каждого клиента;
* сформировать типичные портреты клиентов: выделить несколько наиболее ярких групп и охарактеризовать их основные свойства;
* проанализировать основные признаки, наиболее сильно влияющие на отток;
* сформулировать основные выводы и разработать рекомендации по повышению качества работы с клиентами: 
  * 1) выделить целевые группы клиентов;
  * 2) предложить меры по снижению оттока;
  * 3) определить другие особенности взаимодействия с клиентами.

**Исходные данные.**
    
Набор данных включает следующие поля:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита



## Шаг 1. Откроем файл с данными и изучим общую информацию

In [ ]:
#pip install pymystem3

In [2]:
# imports
import pandas as pd
from pymystem3 import Mystem

In [3]:
data = pd.read_csv('data.csv')

# общая информация о таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ознакомимся с данными (в конце в этой таблице более показательные)

In [4]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

При первом взгляде данные строго типизированны и имеют тип соответствующий содержимому. Не совсем понятны отрицательные значения в столбце 'days_employed' и очень большие положительные цифры у пенсионеров. В столбце 'education' данные в разном регистре.  

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# колиество пропусков (NaN или None) в каждом столбце
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

Как видим пропуски есть только в 2х колонках: 'days_employed' и 'total_income' и их одинаковое количество. Логично предположить, что пропуски в 'days_employed' на тех же строках что и в 'total_income'. Но всё же проверим это

In [6]:
# проверим что пропуски в 'days_employed' на тех же строках что и в 'total_income' 
locs_with_nan = data[data['days_employed'].isna()]
print('Количество строк с пропуском только в одном столбце:', len(locs_with_nan[locs_with_nan['total_income'].notna()]))

Количество строк с пропуском только в одном столбце: 0


In [7]:
# также посчитаем процент пропусков от общего числа строк
print("{:.2%}".format(len(locs_with_nan) / len(data)))

10.10%


**Вывод**

Предположение оказалось верным - каждому пропуску в 'days_employed' соответствует пропуск в total_income.
Тип данных в обоих столбцах 'float'.

Отсюда возможные причины появления пропусков:
* человеческий фактор: при вводе данных был несоблюден формат ввода 'float' (например введен в ковычках или с запятой/точкой вместо точки/запятой в качестве разделителя целой и дробной части), то есть кто-то из сотрудников неверно осведомлен о формате ввода данных столбцов
* программная: возможно, данные попадают в базу из разных модулей и эти строки приходили в БД из модуля, который имеет ошибку в формировании значений.

Процент пропусков относительно большой - больше 10%, поэтому попробуем заполнить данные. Пропуски можно заполнить средними значениями по группам: income_type и purpose. Для более точных результатов заполнять будем уже после категоризации значений из столбца 'purpose'.

### Предобработка данных


In [8]:
# выведем уникальные значения по всем столбцам - посмотрим есть ли какие-то аномалии
for col in data.columns:    
    values = sorted(data[col].unique())
    # если больше 10 значений, выведем только первые и последние 5
    if len(values) > 10:
        display(col.upper(), values[:5], values[-5:])
    else: display(col.upper(), values)
        

'CHILDREN'

[-1, 0, 1, 2, 3, 4, 5, 20]

'DAYS_EMPLOYED'

[-18388.949900568383,
 -17615.563265627912,
 -16593.472817263817,
 -15835.725774811905,
 -15785.678893355003]

[401663.8500458008,
 401674.4666333656,
 401675.093433862,
 401715.8117488882,
 401755.40047533]

...

[20667.26379327158,
 24457.666662383825,
 29154.025210694654,
 29426.692554024463,
 29749.813177053536]

[1711309.267674351,
 1715018.3928324457,
 1726276.0143316735,
 2200852.210258896,
 2265604.028722744]

'PURPOSE'

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье']

['сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недвижимости',
 'сыграть свадьбу']

In [9]:
# подозрительно большие максимальные значения в 'days_employed', проверим:
print('Максимальный стаж: {:8.0f} лет'.format(data['days_employed'].max() / 365))

Максимальный стаж:     1101 лет


**Вывод**

Выявлены следующие аномалии:
* в столбце 'days_employed' вещественный тип, а должен быть целочисленный
* в столбце 'children' есть отрицательное значение
* в столбце 'days_employed' странно записаны данные: часть из них отрицательные, часть положительные
* также в столбце 'days_employed' положительные значения слишком велики (похоже записаны в часах)
* в столбце 'education' значения записаны в разном регистре
* в столбце 'purpose' есть похожие записи, написанные в разных падежах. Нужно произвести лемматизацию для лучшего выделения групп



In [10]:
# поделим положительный стаж на 24 (как выраженный в часах)
data.loc[data['days_employed'] > 0, 'days_employed' ] = data['days_employed'] / 24

# заменим отрицательные на положительные
data['children'] = abs(data['children'])
data['days_employed'] = abs(data['days_employed'])

# заменим регистр в 'education'
data['education'] = data['education'].str.lower()

display(data['children'].unique())
display(data['education'].unique())
print('Минимальный стаж: {:8.0f} дней'.format(data['days_employed'].min()))
print('Максимальный стаж: {:8.0f} лет'.format(data['days_employed'].max() / 365))

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Минимальный стаж:       24 дней
Максимальный стаж:       50 лет


**Вывод**

Отрицательные значения заменены, регистр выровнен, стаж нормализован.

### Обработка дубликатов

In [11]:
print("Явных дубликатов:", data.duplicated().sum())

Явных дубликатов: 71


In [12]:
# удалим явные дубликаты 
data = data.drop_duplicates().reset_index(drop=True)
print("Теперь явных дубликатов:", data.duplicated().sum())

Теперь явных дубликатов: 0


**Оставим только нужные колонки**

In [13]:
data.groupby(['family_status_id', 'family_status'])['total_income'].count()

family_status_id  family_status        
0                 женат / замужем          11143
1                 гражданский брак          3735
2                 вдовец / вдова             865
3                 в разводе                 1083
4                 Не женат / не замужем     2525
Name: total_income, dtype: int64

Как видно кол-во уникальных 'family_status_id' соответствует уникальным 'family_status', значит для нашей задачи можем 'family_status_id' не использовать

In [14]:
# теперь оставим только нужные колонки
data = data[['children', 'dob_years', 'family_status', 'income_type', 'debt', 'total_income', 'purpose']]
data.head()

,children,dob_years,family_status,income_type,debt,total_income,purpose
0,1,42,женат / замужем,сотрудник,0,253875.639453,покупка жилья
1,1,36,женат / замужем,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,женат / замужем,сотрудник,0,145885.952297,покупка жилья
3,3,32,женат / замужем,сотрудник,0,267628.550329,дополнительное образование
4,0,53,гражданский брак,пенсионер,0,158616.077870,сыграть свадьбу


In [15]:
data
m = Mystem()

data

,children,dob_years,family_status,income_type,debt,total_income,purpose
0,1,42,женат / замужем,сотрудник,0,253875.639453,покупка жилья
1,1,36,женат / замужем,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,женат / замужем,сотрудник,0,145885.952297,покупка жилья
3,3,32,женат / замужем,сотрудник,0,267628.550329,дополнительное образование
4,0,53,гражданский брак,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...
21449,1,43,гражданский брак,компаньон,0,224791.862382,операции с жильем
21450,0,67,женат / замужем,пенсионер,0,155999.806512,сделка с автомобилем
21451,1,38,гражданский брак,сотрудник,1,89672.561153,недвижимость
21452,3,38,женат / замужем,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

Дубликаты удалили. Хоть у нас и нет точного идентификатора клиента, но по совокупности данных из всех колонок слишком вероятно, что одинаковые строки являются именно дубликатами.

### Лемматизация

In [ ]:
# проведем лемматизацию колонки 'purpose'
m = Mystem() 

def get_lemmated_text(text):
    try:
        return ''.join(m.lemmatize(text)[:-1]) # [:-1] - это чтобы не брать '\n' в конце
    except: return ''

data['purpose'] = data['purpose'].apply(get_lemmated_text)
data['purpose'].head()

Готово, теперь можно произвести категоризацию.

### Категоризация данных

Для категоризации по доходу напишем метод. Числа категорий были выбраны подбором по примерно сопоставимому кол-ву клиентов в каждой группе и удобству восприятия круглых чисел. Применим метод позже, когда заполним пропущенные значения в 'total_income'.

In [ ]:
# метод для категоризации по доходу
def total_income_group(income_val):    
    if income_val < 100000:
        return 'до 100000'
    if income_val < 150000:
        return 'от 100000 до 150000'
    if income_val < 200000:
        return 'от 150000 до 200000'
    if income_val < 300000:
        return 'от 200000 до 300000'
    else: return 'от 300000'    

Каталогизируем по столбцу 'purpose'. Для начала посмотрим какие категории там есть:

In [ ]:
data['purpose'].unique()

**Вывод**

Прочитав все названия можно заметно выделить несколько категорий: жилье/недвижимость, автомобиль, свадьба, образование.

Теперь напишем метод и выделим эти категории

In [ ]:
# метод для выделения категорий в столбце 'purpose'
def set_purpose_category(purpose):
    if 'жилье' in purpose or 'недвижимость' in purpose:
        return 'жилье/недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'образование' in purpose:
            return 'образование'
    else: return 'другое'
    
    
data['purpose'] = data['purpose'].apply(set_purpose_category)
data['purpose'].value_counts()      

Теперь, имея категории, можем лучше заполнить пропуски в столбце 'total_income'.
Заполним медианой по группам 'purpose' в группах 'income_type'. Если останутся незаполненные заполним медианой только из группы 'income_type'.

In [ ]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'purpose'])['total_income'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

data[data['total_income'].isna()]

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# тут вспомогательные методы

# метод возвращает DataFrame в котором по указанному полю выводится
# отношение клиентов с задолжностью/к клиентам без задолности по кредиту  
def get_debt_ratio_df(groupby_field):
    debt0 = data[data['debt'] == 0].groupby(groupby_field)['debt'].count()
    debt1 = data[data['debt'] == 1].groupby(groupby_field)['debt'].count()

    result_table = (debt1 / debt0).sort_values()

    df = pd.DataFrame(result_table)
    df.rename(columns = {'debt' : 'ratio_debt0_to_debt1'}, inplace=True)  
    return df


# метод для отображения максимального расхождения между минимальным и максимальным значениями
def show_max_difference(selection):
    result = 100 - (100 * (selection.min() / selection.max()))
    display('Максимальное расхождение между минимумом и максимумом: {:8.3}%'.format(result))
    return result

In [ ]:
display('Количество клиентов с данным ко-вом детей', data.groupby('children')['debt'].value_counts())
result_table = get_debt_ratio_df('children')
display(result_table)
selection = result_table['ratio_debt0_to_debt1']
child_diff = show_max_difference(selection)

**Вывод**

Наблюдается следующая зависимость: самые невозвращающие клиенты те у кого 20 детей(возможно тут цифрой 20 выделены все, у кого больше 5 детей), самые возвращающие те у кого нет детей или 3 ребенка. Для тех, у кого 5 детей данных для статистики еще недостаточно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
display('Количество клиентов с данным семейным положением', pd.DataFrame(data.groupby('family_status')['debt'].value_counts()))

display(get_debt_ratio_df('family_status'))

selection = get_debt_ratio_df('family_status')['ratio_debt0_to_debt1']
family_diff = show_max_difference(selection)

**Вывод**

Интересная зависимость: самые необязательные клиенты те, кто никогда не был женат/замужем либо живут в гражданском браке. Самые же обязательные, те кто успел пожить в браке, но овдовел либо развелся.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# применим категоризацию по доходу
data['total_income_groups'] = data['total_income'].apply(total_income_group)
# выводим, чтобы сразу видеть количество
display('Количество клиентов с данным уровнем дохода', data.groupby('total_income_groups')['debt'].count())

display(get_debt_ratio_df('total_income_groups'))

selection = get_debt_ratio_df('total_income_groups')['ratio_debt0_to_debt1']
income_diff = show_max_difference(selection)

**Вывод**

Зависимость выходит такой: меньше всего задолжностей у клиентов с доходом от 200 000 р., далее по надежности идут клиенты с доходом до 100 000 р. А вот менее всего возвращают с доходом от 100 000 р. до 200 000 р.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# проверим количество в каждой категории
display('Количество клиентов с данной целевой категорией', data.groupby('purpose')['debt'].count().sort_values())

# выведем итоговую таблицу
display(get_debt_ratio_df('purpose'))
selection = get_debt_ratio_df('purpose')['ratio_debt0_to_debt1']
purpose_diff = show_max_difference(selection)

**Вывод**

Интересно, что те кто берут деньги на жилье/недвижимость и на свадьбу охотнее возвращают деньги, чем те кто берет их на образование и автомобили.

In [ ]:
# суммарно выведем расхождения между мин и макс по каждой гипотезе
print('Семейное положение:', family_diff)
print('Кол-во детей в семье:', child_diff)
print('Цель получения кредита:', purpose_diff)
print('Ежемесячный доход:', income_diff)

## Шаг 4. Общий вывод

Мы проверили четыре гипотезы и установили что присутствует влияние по каждой из них в таком порядке (по убыванию):
* Семейное положение наиболее влияет на возврат долгов (до **35%** разницы между разными группами )
* Чуть меньше влияет количество детей в семье (до **30%** разницы между разными группами )
* Еще чуть меньше важна цель получени кредита (до **24%** разницы между разными группами )
* И, как ни странно, на последнем месте оказался ежемесячный доход. Этот показатель на последнем месте, но все также значительно видна зависимость и от него. (до **20%** разницы между разными группами )

Из данных выводов можно предположить, что наиболее удобный клиент такой: 
человек без детей, овдовевший либо разведенный, получающий от 200000 т.р., покупающий жиль/недвижимость
А вот тот с кем лучше дела не иметь в плане выдачи кредита:
многодетный человек, ни разу не состоящий в браке, с доходом между 100000 и 200000 т.р., 
собирающийся купить автомобиль или получить образование.







